In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter

from sklearn import datasets
from sklearn.model_selection import train_test_split

In [2]:
def fcnCalculateEntropy(y):
    _, yUnique = np.unique(y, return_counts=True)
    lstProbability = yUnique / len(y)
    return -np.sum([pi * np.log2(pi) for pi in lstProbability if pi > 0])

In [3]:
class Node:

    def __init__(self, feature = None, threshold = None, left = None, right = None, *, value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        
    def is_leaf_node(self):
        return self.value is not None
    
class C_4_5_DecisionTreeClassifier:
    
    def __init__(self, min_sample_split = 2, max_depth = 100, n_feats = None, threshold_for_category = 10):
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None
        self.FeatureType = None
        self.threshold_for_category = threshold_for_category
        self.y = None
        
    def _label_categorical_features(self, y):
        yUnique = np.unique(y)
        Y = np.zeros(len(y) , dtype = int)
        for intCtr in range(0, len(y)):
            Y[intCtr] = np.argwhere(y[intCtr] == yUnique)
        
        return Y

    def _determine_feature_type(self, X):
        lstFeatureType = []
        for intFeatureIterator in range(0, X.shape[1]):
            if len(np.unique(X[:, intFeatureIterator])) <= self.threshold_for_category or isinstance(np.unique(X[:, intFeatureIterator]), str):
                lstFeatureType.append("Categorical")
            else:
                lstFeatureType.append("Continuous")
        
        return lstFeatureType
        
    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(X.shape[1], self.n_feats)
        self.FeatureType = self._determine_feature_type(X)
        self.y = self._label_categorical_features(y)
        self.root = self._grow_tree(X, y)
        
    def _grow_tree(self, X, y, depth = 0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_sample_split):
            leaf_value = self._most_common_label(y)
            return Node(value = leaf_value)
        
        feat_idxs = np.random.choice(n_features, self.n_feats, replace = False)
        
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._Split(X[:, best_feat], best_thresh, self.FeatureType[best_feat])
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        
        return Node(best_feat, best_thresh, left, right)       
    
    def _best_criteria(self, X, y, feat_idxs):
        best_gain_ratio = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            feature_type = self.FeatureType[feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain_ratio = self._gain_ratio(self.y, X_column, threshold, feature_type)
                if gain_ratio > best_gain_ratio:
                    best_gain_ratio = gain_ratio
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh
    
  
    def _gain_ratio(self, y, X_column, split_thresh, feature_type):
        parent_entropy = fcnCalculateEntropy(y)
        left_idxs, right_idxs = self._Split(X_column, split_thresh, feature_type)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = fcnCalculateEntropy(y[left_idxs]), fcnCalculateEntropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r
        gain_split = parent_entropy - child_entropy
        split_info = fcnCalculateEntropy(X_column)
        return gain_split / split_info
         
    
    def _Split(self, X_column, split_thresh, feature_type):
        if feature_type == "Continuous":
            left_idxs = np.argwhere(X_column <= split_thresh).flatten()
            right_idxs = np.argwhere(X_column > split_thresh).flatten()
        else:
            left_idxs = np.argwhere(X_column == split_thresh).flatten()
            right_idxs = np.argwhere(X_column != split_thresh).flatten()            
        return left_idxs, right_idxs
        
    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, X, node):
        if node.is_leaf_node():
            return node.value

        if self.FeatureType[node.feature] == "Categorical":
            if X[node.feature] == node.threshold:
                return self._traverse_tree(X, node.left)
            return self._traverse_tree(X, node.right)        
        else:
            if X[node.feature] <= node.threshold:
                return self._traverse_tree(X, node.left)

            return self._traverse_tree(X, node.right)

def fcnAccuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [4]:
data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

In [5]:
dt = C_4_5_DecisionTreeClassifier(max_depth = float('inf'), threshold_for_category=0)
dt.fit(X_train, y_train)
    
y_pred = dt.predict(X_test)
accuracy = fcnAccuracy(y_test, y_pred)

print ("Accuracy:", accuracy)

Accuracy: 0.9230769230769231


In [8]:
data2 = datasets.load_iris()
X2 = data2.data
y2= data2.target

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.25, random_state=2)

In [15]:
dt4 = C_4_5_DecisionTreeClassifier(max_depth=100, threshold_for_category=0)
dt4.fit(X2_train, y2_train)
    
y_pred4 = dt4.predict(X2_test)
accuracy4 = fcnAccuracy(y2_test, y_pred4)

print ("Accuracy:", accuracy4)

Accuracy: 0.868421052631579
